# Async Agentic Workflow with Databricks Lakeflow Jobs

This notebook demonstrates how to use Databricks Lakeflow Jobs to execute async agentic workflows.

**Key Demo Points:**
- Interactive agent for research planning (Planner Agent)
- Async job execution via Lakeflow (Researcher Agent)
- Non-blocking polling for job status
- Results saved to Unity Catalog Volume

## Flow
1. Converse with Planner Agent to create a research plan
2. Approve the plan → triggers async Lakeflow Job
3. Continue working while job runs (poll for updates as needed)
4. Retrieve completed report from UC Volume

## Setup

In [ ]:
%pip install databricks-sdk databricks-mcp openai pydantic pyyaml --quiet
dbutils.library.restartPython()

In [ ]:
import sys
import os
import yaml

# Auto-detect src path from notebook location
# Notebook is in src/notebooks/, so src is ../
SRC_PATH = os.path.abspath(os.path.join(os.getcwd(), ".."))
print(f"Auto-detected SRC_PATH: {SRC_PATH}")

sys.path.insert(0, SRC_PATH)

# Load configuration from config.yaml
config_path = os.path.join(SRC_PATH, "config.yaml")
with open(config_path, "r") as f:
    config = yaml.safe_load(f)

print(f"Loaded config from: {config_path}")

# Verify imports work
from models.research_plan import ResearchPlan
from planner_agent import PlannerAgent
from job_tools import check_job_status

print("Imports successful!")

## Configuration

Update these values for your environment:

In [ ]:
# ============================================
# CONFIGURATION - Loaded from config.yaml
# ============================================

# LLM endpoint (Databricks Foundation Model)
LLM_ENDPOINT = config["llm"]["endpoint_name"]

# Path to the researcher notebook in your workspace
RESEARCHER_NOTEBOOK_PATH = config["paths"]["researcher_notebook"]

# UC Volume path for output reports
OUTPUT_VOLUME_PATH = config["paths"]["output_volume"]

print(f"LLM Endpoint: {LLM_ENDPOINT}")
print(f"Researcher Notebook: {RESEARCHER_NOTEBOOK_PATH}")
print(f"Output Volume: {OUTPUT_VOLUME_PATH}")

## Initialize Planner Agent

In [ ]:
from databricks.sdk import WorkspaceClient

ws = WorkspaceClient()
print(f"Connected to: {ws.config.host}")

agent = PlannerAgent(
    llm_endpoint=LLM_ENDPOINT,
    researcher_notebook_path=RESEARCHER_NOTEBOOK_PATH,
    output_volume_path=OUTPUT_VOLUME_PATH,
    workspace_client=ws,
)

print("Planner Agent initialized!")

## Interactive Conversation

Run the cell below to start an interactive conversation with the Planner Agent.

**Conversation flow:**
1. Describe your research topic
2. Agent proposes research questions
3. Refine the questions through conversation
4. Say "run it" or "execute" when ready → Agent submits async Lakeflow Job
5. Type "quit" or "exit" to end the conversation

The Planner Agent can:
- Help you develop research questions
- Submit approved plans to a Lakeflow Job for async execution
- Check job status
- Retrieve completed reports

In [ ]:
# Interactive conversation loop
# Type your messages and press Enter. Type "quit" or "exit" to end.

print("=" * 60)
print("PLANNER AGENT - Interactive Research Planning")
print("=" * 60)
print("Describe your research topic to get started.")
print("Type 'quit' or 'exit' to end the conversation.")
print("=" * 60)
print()

while True:
    try:
        user_input = input("You: ").strip()
    except EOFError:
        break
    
    if not user_input:
        continue
    
    if user_input.lower() in ["quit", "exit", "q"]:
        print("\nEnding conversation. Active jobs will continue running.")
        break
    
    # Send to agent and get response
    response = agent.chat(user_input)
    print(f"\nAgent: {response}\n")

## After Exiting the Conversation

Once you exit the interactive loop, you can still:
- Check job status with `agent.get_active_jobs()`
- Resume conversation with `agent.chat("your message")`
- Reset conversation with `agent.reset_conversation()`

In [ ]:
# Check status of all active jobs
for job in agent.get_active_jobs():
    print(f"Run ID: {job['run_id']}")
    print(f"  Topic: {job['topic']}")
    print(f"  State: {job['state']}")
    print(f"  Output: {job['output_path']}")
    print()

## Retrieve Results

Once the job completes, retrieve the research report:

In [ ]:
# Ask the agent to retrieve the report
# Only works after job completes
response = agent.chat("Show me the research report")
print(response)

In [ ]:
# Alternative: Read directly from UC Volume
# Replace with actual output path
# output_path = "/Volumes/{catalog}/{schema}/{volume}/report_xxx.md"
# with open(output_path, 'r') as f:
#     report = f.read()
# print(report)

## Key Takeaways

This demo shows:

1. **Async Execution**: The Planner Agent kicks off a Lakeflow Job and returns immediately - it doesn't wait.

2. **Long-Running Tasks**: The Researcher Agent can run for >5 minutes (up to the job timeout) without blocking.

3. **Non-Blocking Polling**: Check job status anytime without waiting for completion.

4. **UC Volume Output**: Results are persisted to Unity Catalog Volume for reliable retrieval.

5. **MCP Tool Integration**: The Researcher Agent uses Databricks MCP for web search capabilities.